In [56]:
import os
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout, Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [57]:
# Load and preprocess medical anomaly data
medical_image_path = 'D:/PBL2_project/medical_dataset/'
medical_datagen = ImageDataGenerator(rescale=1./255)
medical_image_generator = medical_datagen.flow_from_directory(
    medical_image_path,
    target_size=(224, 224),
    batch_size=5,  # Set batch size to 14
    class_mode='binary'
) 

Found 495 images belonging to 1 classes.


In [58]:
# Load and preprocess traffic anomaly data
traffic_image_path = 'D:/PBL2_project/Dataset/'
traffic_datagen = ImageDataGenerator(rescale=1./255)
traffic_image_generator = traffic_datagen.flow_from_directory(
    traffic_image_path,
    target_size=(224, 224),
    batch_size=5,  # Set batch size to 14
    class_mode='binary'
)

Found 985 images belonging to 2 classes.


In [59]:
# Define the architecture for the medical anomaly detection model
input_medical = Input(shape=(224, 224, 3))
base_model_medical = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=input_medical)
x_medical = base_model_medical.output
x_medical = tf.keras.layers.GlobalAveragePooling2D()(x_medical)
output_medical = Dense(1, activation='sigmoid')(x_medical)
model_medical = Model(inputs=input_medical, outputs=output_medical)

In [60]:
# Define the architecture for the traffic anomaly detection model
input_traffic = Input(shape=(224, 224, 3))
base_model_traffic = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_traffic)
x_traffic = base_model_traffic.output
x_traffic = tf.keras.layers.GlobalAveragePooling2D()(x_traffic)
output_traffic = Dense(1, activation='sigmoid')(x_traffic)
model_traffic = Model(inputs=input_traffic, outputs=output_traffic)


In [61]:
# Concatenate the output features of both models
concatenated = Concatenate()([model_medical.output, model_traffic.output])

In [62]:
# Define a meta-classifier to make final predictions
meta_classifier = Dense(1, activation='sigmoid')(concatenated)
combined_model = Model(inputs=[input_medical, input_traffic], outputs=meta_classifier)


In [63]:
# Compile the combined model
combined_model.compile(optimizer=Adam(learning_rate=0.001),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])


In [64]:
# Train the combined model using the custom generator
combined_model.fit(combined_generator(medical_image_generator, traffic_image_generator), steps_per_epoch=len(medical_image_generator), epochs=20)

Epoch 1/20
99/99 [==============================] - 155s 1s/step - loss: 0.2528 - accuracy: 0.9960
Epoch 2/20
99/99 [==============================] - 133s 1s/step - loss: 0.2090 - accuracy: 1.0000
Epoch 3/20
99/99 [==============================] - 131s 1s/step - loss: 0.1787 - accuracy: 1.0000
Epoch 4/20
99/99 [==============================] - 131s 1s/step - loss: 0.1540 - accuracy: 1.0000
Epoch 5/20
99/99 [==============================] - 5318s 54s/step - loss: 0.1339 - accuracy: 1.0000
Epoch 6/20
99/99 [==============================] - 132s 1s/step - loss: 0.1172 - accuracy: 1.0000
Epoch 7/20
99/99 [==============================] - 131s 1s/step - loss: 0.1033 - accuracy: 1.0000
Epoch 8/20
99/99 [==============================] - 131s 1s/step - loss: 0.0917 - accuracy: 1.0000
Epoch 9/20
99/99 [==============================] - 131s 1s/step - loss: 0.0817 - accuracy: 1.0000
Epoch 10/20
99/99 [==============================] - 278s 3s/step - loss: 0.0733 - accuracy: 1.0000
Epoch 1

In [65]:
# Save the combined model
combined_model.save('combined_model.h5')


c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
